# Pole stiction

This notebook is intended to characterize the slow speed stiction problem \
at the pole by spiraling into the pole where the tracking speeds will be slower and slower. \
Craig Lage - 17-Oct-22

In [ ]:
import asyncio
import time
import os
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time

from lsst.ts import salobj

from lsst.ts.observatory.control.auxtel.atcs import ATCS
from lsst.ts.observatory.control.auxtel.latiss import LATISS
from lsst.ts.observatory.control.utils import RotType

In [ ]:
logger = logging.getLogger(f"Daytime Checkout {Time.now()} UT")
logger.level = logging.DEBUG

In [ ]:
logger.info(os.environ["OSPL_URI"])
logger.info(os.environ["LSST_DDS_PARTITION_PREFIX"])

### Instantiate the control classes

In [ ]:
domain = salobj.Domain()
atcs = ATCS(domain)
latiss = LATISS(domain)
await asyncio.gather(atcs.start_task, latiss.start_task)
await atcs.assert_liveliness()

## Enable LATISS and ATCS

In [ ]:
# Didn't come up.  Brought up with LOVE
await latiss.enable()
await atcs.enable()
await atcs.assert_all_enabled()

If ATCamera and/or ATOODS don't come up, see section `Quick Troubleshooting` for support commands. 

### Bias verification

In [ ]:
for i in range(5):
    await latiss.take_bias(nbias=1)
    await asyncio.sleep(2.0)

In [ ]:
await atcs.disable_dome_following()

### Plot planned locations in RA/Dec space

In [ ]:
%matplotlib inline
ra = 12.0
deltaRa = 2.5
logCosDec = -0.8
deltaLogCosDec = -0.1
thetas = []
decs = []
for i in range(22):
    theta = ra * 15.0
    dec = - np.arccos(10**logCosDec) * 180.0 / np.pi
    #print(logCosDec, dec, 90.0-dec)
    thetas.append(theta / 180.0 * np.pi)
    decs.append(90.0 + dec)
    #print(theta, 90.0-dec)
    ra += deltaRa
    logCosDec += deltaLogCosDec
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(thetas, decs, marker = 'x')
ax.set_rmax(10.0)
ax.set_rticks([5.0, 10.0])  # Less radial ticks
#ax.set_rlabel_position(-22.5)  # Move radial labels away from plotted line
ax.grid(True)

ax.set_title("Planned Stiction Points\nR shows distance from pole in degrees\n 40 points total")
plt.show()

### Now run the actual test, taking a 30 second image at each location.

In [ ]:
# Move to starting location
ra = 12.0
logCosDec = -0.8
start_rot = 0.0
dec = - np.arccos(10**logCosDec) * 180.0 / np.pi
print(ra, dec)
await atcs.slew_icrs(ra, dec, start_rot)

In [ ]:
await latiss.take_engtest(30.0, filter='empty_1',grating='empty_1')

In [ ]:
# Now run the actual test
# First image is 2023032100008
# ATPtg faulted at seqNum 23 for rotator out of range
# Moving on to reverse direction, setting rotator to physical 0
# This will speed things up.
ra = 12.0
deltaRa = 2.5
logCosDec = -0.8
deltaLogCosDec = -0.1
start_rot = 0.0
for i in range(22):
    dec = - np.arccos(10**logCosDec) * 180.0 / np.pi
    ra += deltaRa
    logCosDec += deltaLogCosDec
    await atcs.slew_icrs(ra, dec, rot=start_rot, rot_type=RotType.Physical)
    await latiss.take_engtest(30.0, filter='empty_1',grating='empty_1')


### Plot planned locations in RA/Dec space - reverse direction

In [ ]:
ra = 12.0
deltaRa = 2.5
logCosDec = -0.8
deltaLogCosDec = -0.1
thetas = []
decs = []
for i in range(22):
    theta = ra * 15.0
    dec = - np.arccos(10**logCosDec) * 180.0 / np.pi
    #print(logCosDec, dec, 90.0-dec)
    thetas.append(theta / 180.0 * np.pi)
    decs.append(90.0 + dec)
    #print(theta, 90.0-dec)
    ra -= deltaRa
    logCosDec += deltaLogCosDec
fig, ax = plt.subplots(subplot_kw={'projection': 'polar'})
ax.plot(thetas, decs, marker = 'x')
ax.set_rmax(10.0)
ax.set_rticks([5.0, 10.0])  # Less radial ticks
#ax.set_rlabel_position(-22.5)  # Move radial labels away from plotted line
ax.grid(True)

ax.set_title("Planned Stiction Points\nR shows distance from pole in degrees\n 40 points total")
plt.show()

### Now run the actual test, taking a 30 second image at each location. Reverse direction

In [ ]:
# Move to starting location
ra = 12.0
logCosDec = -0.8
start_rot = 0.0
dec = - np.arccos(10**logCosDec) * 180.0 / np.pi
print(ra, dec)
await atcs.slew_icrs(ra, dec, start_rot, rot_type=RotType.Physical)

In [ ]:
# Now run the actual test
# First image is 2023032100024
ra = 12.0
deltaRa = 2.5
logCosDec = -0.8
deltaLogCosDec = -0.1
start_rot = 0.0
for i in range(22):
    dec = - np.arccos(10**logCosDec) * 180.0 / np.pi
    ra -= deltaRa
    logCosDec += deltaLogCosDec
    await atcs.slew_icrs(ra, dec, rot=start_rot, rot_type=RotType.Physical)
    await latiss.take_engtest(30.0, filter='empty_1',grating='empty_1')


### Now run the actual test, taking a 30 second image at each location.
### Reverse direction.  Fan on @ 40 Hz.

In [ ]:
# Move to starting location
ra = 12.0
logCosDec = -0.8
start_rot = 0.0
dec = - np.arccos(10**logCosDec) * 180.0 / np.pi
print(ra, dec)
await atcs.slew_icrs(ra, dec, start_rot, rot_type=RotType.Physical)

In [ ]:
# Now run the actual test
# First image is 2023032100046
# Re-starting after ATCamera fault
# First image is 2023032100054
# Need to run darks, engtests are saturating
ra = 12.0
deltaRa = 2.5
logCosDec = -0.8
deltaLogCosDec = -0.1
start_rot = 0.0
for i in range(22):
    dec = - np.arccos(10**logCosDec) * 180.0 / np.pi
    ra -= deltaRa
    logCosDec += deltaLogCosDec
    await atcs.slew_icrs(ra, dec, rot=start_rot, rot_type=RotType.Physical)
    await latiss.take_darks(30.0, 1)


In [ ]:
# Stop tracking
await atcs.stop_tracking()

## Shutdown all ATCS components

In [ ]:
await atcs.shutdown()

## Put LATISS in standby

In [ ]:
await latiss.standby()